In [0]:
import pandas as pd
import numpy as np

# Importing Dataset


In [0]:
data = pd.read_csv('reviews.csv')

In [0]:
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [0]:
data.shape

(67986, 8)

In [0]:
data['rating'].unique()

array([3, 1, 5, 4, 2])

In [0]:
data.insert(8,"output","")

In [0]:
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes,output
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,


# Classifying the review column into Positive and Negative(1/0)

In [0]:
for i in range(0,len(data['rating'])):
  num=data['rating'][i]
  if num>3:
    data['output'][i]=1
  else:
    data['output'][i]=0  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes,output
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,1
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,1


In [0]:
data['review']=data['title']+data['body']

In [0]:
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes,output,review
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,0,"Def not best, but not worstI had the Samsung A..."
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,0,Text Messaging Doesn't WorkDue to a software i...
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,1,"Love This PhoneThis is a great, reliable phone..."
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,0,"Love the Phone, BUT...!I love the phone and al..."
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,1,"Great phone service and options, lousy case!Th..."


In [0]:
data=data.drop(["title","body"],axis=1)

In [0]:
data.head()

,asin,name,rating,date,verified,helpfulVotes,output,review
0,B0000SX2UC,Janet,3,"October 11, 2005",False,1.0,0,"Def not best, but not worstI had the Samsung A..."
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,17.0,0,Text Messaging Doesn't WorkDue to a software i...
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,5.0,1,"Love This PhoneThis is a great, reliable phone..."
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,1.0,0,"Love the Phone, BUT...!I love the phone and al..."
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,1.0,1,"Great phone service and options, lousy case!Th..."


In [0]:
y=data.iloc[:,6:7].values

In [0]:
y.shape

(67986, 1)

# Importing Libraries for NLP

In [0]:
import re
import nltk#natural language tool kit
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
ps=PorterStemmer()

In [0]:
data['review'].isnull().any()

True

In [0]:
data['review'].fillna(data['review'].mode()[0],inplace=True)

In [0]:
data['review'].isnull().any()

False

In [0]:
list1=[]
for i in range(len(data['review'])):
    review=data["review"][i]
    review=re.sub("[^a-zA-Z]"," ",review)
    review=review.lower()
    review=review.split()    
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=" ".join(review)
    list1.append(review)

# Vectorization

In [0]:
#vectorization
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
x=cv.fit_transform(list1).toarray()

In [0]:
x.shape

(67986, 3000)

In [0]:
!mkdir -p count1
import pickle 
Filename="count1/countvectorizer.pkl"
with open(Filename,"wb") as file:
  pickle.dump(cv,file)

# splitting the data into train and test

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [0]:
x_train.shape

(54388, 3000)

# Apply ANN

In [0]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units =3000  ,init = "random_uniform",activation = "relu"))
model.add(Dense(units =3000  ,init = "random_uniform",activation = "relu"))

model.add(Dense(units = 1 ,init = "random_uniform",activation = "sigmoid"))
model.compile(optimizer = "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
model.fit(x_train,y_train,epochs  = 8)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3000, activation="relu", kernel_initializer="random_uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3000, activation="relu", kernel_initializer="random_uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="random_uniform")`
  import sys


Epoch 1/8
54388/54388 [==============================] - 407s 7ms/step - loss: 0.2683 - accuracy: 0.8953
Epoch 2/8
54388/54388 [==============================] - 404s 7ms/step - loss: 0.1539 - accuracy: 0.9417
Epoch 3/8
54388/54388 [==============================] - 409s 8ms/step - loss: 0.0678 - accuracy: 0.9769
Epoch 4/8
54388/54388 [==============================] - 407s 7ms/step - loss: 0.0340 - accuracy: 0.9899
Epoch 5/8
54388/54388 [==============================] - 405s 7ms/step - loss: 0.0265 - accuracy: 0.9924
Epoch 6/8
54388/54388 [==============================] - 412s 8ms/step - loss: 0.0185 - accuracy: 0.9942
Epoch 7/8
54388/54388 [==============================] - 415s 8ms/step - loss: 0.0221 - accuracy: 0.9943
Epoch 8/8
54388/54388 [==============================] - 416s 8ms/step - loss: 0.0220 - accuracy: 0.9943


In [0]:
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)
y_pred

array([[ True],
       [ True],
       [False],
       ...,
       [ True],
       [False],
       [ True]])

In [0]:
y_test

array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=object)

In [0]:
y_p=model.predict(cv.transform(['bad phone']))
y_p

array([[0.36412725]], dtype=float32)

In [0]:
y_p>0.5

array([[False]])

In [0]:
model.save("phone_review.h5")